# Transform to Load into DDBB (2)

I need to transform the datasets to adapt and load them into a DDBB

0. Import modules
1. Load clean dataframes
2. MERGE AMAZON AND ASOS DATASETS

### 0. Import modules

In [30]:
# Import modules etc

import pandas as pd
import numpy as np
import re
import random
from unidecode import unidecode
import re

np.random.seed(42)
pd.set_option('display.max_columns', None) # show all the columns

import warnings
warnings.filterwarnings('ignore') # ignorar warnings

from src import dataanalysis_fun1 as mlg # Import my module

####
# import mysql.connector as conn
# from sqlalchemy import create_engine

## 1. Load clean dataframes

In [31]:
kaggle_clean = pd.read_csv('../data/clean/kaggle_clean.csv')
asos_womenupperclothes_clean = pd.read_csv('../data/clean/asos_womenupperclothes_clean.csv')
amz1_womentshirt_clean = pd.read_csv('../data/clean/amz1_womentshirt_clean.csv')

In [32]:
asos_womenupperclothes_clean["shop"]="asos"
asos_womenupperclothes_clean.head()

,brand,description,price,colour,shop
0,New Look,Top negro escalonado de manga larga con estamp...,16.99,negro,asos
1,Mamalicious,Top color ámbar dorado de manga larga con cuel...,45.99,dorado,asos
2,Mamalicious,Top amarillo de manga larga con cuello ancho d...,21.99,dorado,asos
3,Selected,Top marrón de manga larga con cuello alto de S...,31.99,marron,asos
4,Mamalicious,Top multicolor a rayas con cuello alto de punt...,40.99,multicolor,asos


In [33]:
amz1_womentshirt_clean["shop"]="amazon"
amz1_womentshirt_clean.head()

,brand,description,price,colour,shop
0,Ranphee,Camisetas Túnica de Manga Larga para Mujer Blu...,25.99,negro,amazon
1,ANGGREK,Mujer Verano Tops Casual Encaje Cuello en V Ca...,31.19,negro,amazon
2,GRMLRPT,2023 Camiseta de Manga 3/4 Mujer Moda Cuello e...,25.99,negro,amazon
3,Urban Classics,Ladies Extended Shoulder tee Camiseta para Mujer,11.90,negro,amazon
4,Only,"Printed 3/4 Sleeved Top Suéter, Nero (Black/AO...",19.99,negro,amazon


## 2. Create 3 DFs `product`,  `costumer` and `sale`

In [34]:
product_df = pd.concat([asos_womenupperclothes_clean, amz1_womentshirt_clean], ignore_index=True)

In [35]:
product_df = product_df.sort_values(by='brand', ascending=True)
product_df["product_id"]=range(1,len(product_df)+1)
product_df.head()

,brand,description,price,colour,shop,product_id
9123,& Other Stories,Vestido corto rosa pastel con diseño bordado y...,54.0,rosa,asos,1
5153,& Other Stories,Vestido corto azul cruzado de mezcla de lino d...,43.5,azul,asos,2
5154,& Other Stories,Vestido midi blanco hueso con estampado de lun...,64.0,negro,asos,3
4741,& Other Stories,Vestido corto verde de corte skater con estamp...,45.0,verde,asos,4
7461,& Other Stories,Vestido corto con estampado a rayas de algodón...,36.5,multicolor,asos,5


### 2.1 Classify products according to their description


In [36]:
product_df1=product_df.copy()
product_df.head()

type_dict={"camiseta":["camiseta", "shirt", "polo", "Camicetta", "manga larga", "camisola", "manica lunga"],
           "top": ["top"],
           "chaqueta":["chaqueta","sudadera", "jersey", "pulover","pullover", "sueter", "polar", "cardigan", "casaca"],
           "vestido":["vestido", "tunica", "camison"],
           "camisa": ["camisa", "blusa", "blouse", "bluse"],
           "lenceria":["lenceria", "sujetador", "corse"],
           "chaleco": ["chaleco"],
          "casacas sanitarias": ["casacas sanitarias", "uniformes sanitarios", "uniforme sanitario", "trabajo"],
           "pantalones": ["pantalones",  "jeans", "vaqueros"],
          "mallas": [ "leggings", "mallas", "joggers", "mallot", "leotardo"],
            "falda": [ "falda"],
                       "zapatos": [ "zapatos"]
          }
                                                                          
def color_dict_simplify(x):
    x = x.lower()
    x=unidecode(x) #Take out accentuation
    theval=""
    for KKK, VVV in type_dict.items():
        for VAL in VVV:
            if VAL.lower() in x:
                theval = KKK
            
    return theval

product_df1["type"] = product_df1['description'].apply(color_dict_simplify)

print(len(product_df1[product_df1["type"]==""]))

134


#### Only 134 products remained without `type` classification

In [37]:
product_df1["type"].value_counts()

vestido               7192
camisa                6700
camiseta              6457
top                   2622
chaleco                648
casacas sanitarias     538
chaqueta               439
pantalones             143
falda                  139
lenceria               137
                       134
mallas                  29
zapatos                  6
Name: type, dtype: int64

### 2.2  Divide price of Packs

In [109]:
product_df2=product_df1.copy()
pack_dict={"pack":["pack", "packs"]}


def subset_pack(x):
    x = x.lower()
    x=unidecode(x)
    theval=1
    for KKK, VVV in pack_dict.items():
        for VAL in VVV:
            if VAL.lower() in x: # the item is actually a pack
                #find the number of elements and divide the price
                digits_list = re.findall(r'\b\d\b', x)

                numeric_values = [int(digit) for digit in digits_list]
                if numeric_values:
                    theval = min(numeric_values)
                    #if theval == 0:
                    #    print(digits_only, x)
                else:
                    theval=1

    return theval

product_df2["price_pack"]=product_df2['description'].apply(subset_pack)


There are 2 rows including the term "pack" but actually being only a single product.<br/>
These have a price_pack = 0, what returns and inf. <br/> Manually modify them to original price

In [110]:
zeroval=product_df2["price_pack"]==0
product_df2.loc[zeroval, 'price_pack'] = 1

product_df2["price_pack"]=product_df2["price"]/product_df2["price_pack"]

nan_inf_mask = product_df2["price_pack"].isin([np.nan, np.inf, -np.inf])
any(nan_inf_mask)

False

## Check for duplicates 

We expect to have the **same product with the same description for different colors**. <br/>
**That is indeed the point of the EDA**, to check whether the color affects to the final product price.

In [118]:
product_complete[product_complete.duplicated()]

,brand,description,price,colour,shop,product_id,type,price_pack


In [119]:
product_complete=product_df2.reset_index(drop=True)
product_complete.index=product_complete.index+1
product_simple=product_complete[["brand", "colour", "shop", "product_id", "type", "price_pack"]]

## 3. Save `products` df

* `complete` info of products: including description and original price
* `simple` info of products: only brand, type, and price divided by pack #


The lines are commented to avoid overwriting

In [120]:
#product_simple.to_csv('../data/clean/product_simple.csv', index=False)
#product_complete.to_csv('../data/clean/product_complete.csv', index=False)

## 4. Create `customer_df` and `sale_df` from `kaggle_clean`


* Kaggle dataset is an artificial set that recreates shopping info.
* We will exploit this data and insert additional data to perform a prototype of ETL/EDA project.

* Kaggle table collects data for **3900 artifical clients**
* **To make it more interesting,**  we will insert in `customer_df` artificial personal info as `horoscope`, `blood type` or wheteher the costumer is a `cat/dog_type` person.

In [ ]:
kaggle_clean.head()

In [175]:
customer_df=kaggle_clean[["customer_id", "age", "gender", "location"]]
customer_df["horoscope"]=""
customer_df["blood"]=""
customer_df["catdog"]=""

In [176]:
horoscope = [
    'Aries',
    'Taurus',
    'Gemini',
    'Cancer',
    'Leo',
    'Virgo',
    'Libra',
    'Scorpio',
    'Sagittarius',
    'Capricorn',
    'Aquarius',
    'Pisces'
]

catdog = [
    'cat',
    'dog',
    "none"
]

blood = [
    'A',
    'B',
    'AB',
    'O'
]


for i in range(len(customer_df)):
    customer_df["horoscope"][i]=random.choice(horoscope)
    customer_df["catdog"][i]=random.choice(catdog)
    customer_df["blood"][i]=random.choice(blood)

In `sales_df` we will introduce randomly the customer info.

We have data for 3900 clients and >25,000 products

* **We must ensure that all clients shop at least one product**
* FOR THE FUTURE Preferably some clients will shop more than one product
* FOR THE FUTURE Some products will not be bought by any customer

In [228]:
sale_df=kaggle_clean[["customer_id", "payment_method", "review_rating", "season"]]
sale_df["product_id"]=""

### 4.1 Add to `sale_df` randomly chosen values of `product_ids` from `product_df`

In [230]:
for i in range(len(sale_df)):
    sale_df["product_id"][i]=random.choice(product_simple.product_id)

#### There are multiple products absent in `sale_df` (i.e., not selled products)

It is expected, it is just an observation

In [232]:
AA=set(sale_df["product_id"]).difference(set(product_simple["product_id"]))
BB=set(product_simple["product_id"]).difference(set(sale_df["product_id"]))
print(len(AA), len(BB))

0 21583


#### All the customers have a sale_id!

In [233]:
AA=set(sale_df["customer_id"]).difference(set(customer_df["customer_id"]))
BB=set(customer_df["customer_id"]).difference(set(sale_df["customer_id"]))
print(len(AA), len(BB))

0 0


## 5.  `customer_df` and `sale_df`

The lines are commented to avoid overwriting

In [235]:
#customer_df.to_csv('../data/clean/customer_df.csv', index=False)
#sale_df.to_csv('../data/clean/sale_df.csv', index=False)

In [248]:
product_simple
#customer_df

,brand,colour,shop,product_id,type,price_pack
1,& Other Stories,rosa,asos,1,vestido,54.00
2,& Other Stories,azul,asos,2,vestido,43.50
3,& Other Stories,negro,asos,3,vestido,64.00
4,& Other Stories,verde,asos,4,vestido,45.00
5,& Other Stories,multicolor,asos,5,vestido,36.50
...,...,...,...,...,...,...
25180,yeyity,blanco,amazon,25180,camisa,42.99
25181,yiouyisheng,amarillo,amazon,25181,camiseta,9.98
25182,yiouyisheng,morado,amazon,25182,top,11.99
25183,yiouyisheng,turquesa,amazon,25183,camisa,8.99


## 6. Save as a single Excel file

In [265]:
# Assuming df1, df2, and df3 are your DataFrames

# Create an Excel writer
excel_writer = pd.ExcelWriter('color.xlsx')

# Write each DataFrame to a different sheet
product_simple.to_excel(excel_writer, sheet_name='product', index=False)
customer_df.to_excel(excel_writer, sheet_name='customer', index=False)
sale_df.to_excel(excel_writer, sheet_name='sale', index=False)

# Save the Excel file
excel_writer.save()


## 7. Load and work with python - mySQL

In [249]:
file_path = "../notebooks/temp/logs/PASSES.py"

# Create an empty namespace (dictionary) to store variables
namespace = {}

# Execute the Python file and populate the namespace
with open(file_path, "r") as reader:
    exec(reader.read(), namespace)

# Access the imported variable
SQL = namespace.get("SQL")

len(SQL)
DDBB="colors"

### SqlAlchemy


* Create EER Diagram in mySQL workbench
* Load data (csv)

In [259]:
str_conn = 'mysql+pymysql://root:' + SQL + '@localhost:3306/'+ DDBB
cursor = create_engine(str_conn)

In [260]:
product_simple.to_sql(name='product',      # nombre de la tabla
            con=cursor,          # conexion al servidor
            if_exists='append',  # reemplaza la tabla si existe
            index=False)

In [261]:

customer_df.to_sql(name='customer',      # nombre de la tabla
            con=cursor,          # conexion al servidor
            if_exists='append',  # reemplaza la tabla si existe
            index=False)



### Change the foreign key colnames

* MySQL Workbench modifies the column names of foreign keys when making the relation with their PKs
* We need to change these names in jupyter too to avoid mismatches

In [253]:
sale_df.head()

,customer_id,payment_method,review_rating,season,product_id
0,1,Venmo,3.1,Winter,18767
1,2,Cash,3.1,Winter,18406
2,3,Credit Card,3.1,Spring,20171
3,4,PayPal,3.5,Spring,2660
4,5,PayPal,2.7,Spring,11151


In [262]:
# Crea un diccionario para mapear nombres de columnas existentes a los nuevos nombres
nuevos_nombres = {'customer_id': 'customer_customer_id', 'payment_method': 'payment_method',
                 'review_rating': 'review_rating', 'season': 'season', 'product_id': 'product_product_id'}

# Utiliza la función rename para cambiar los nombres de las columnas
sale_df = sale_df.rename(columns=nuevos_nombres)

In [263]:

sale_df.to_sql(name='sale',      # nombre de la tabla
            con=cursor,          # conexion al servidor
            if_exists='append',  # reemplaza la tabla si existe
            index=False)

